# 소방청 - 서울지역 검색

In [ ]:
관서명 주소 전화번호
'''https://www.nfa.go.kr/nfa/introduce/
    status/firestationidfo/?searchDistance=10
    &searchMode=distance&myX=37.5382842&myY=126.9213649
    &searchKeyword='''
csv파일로 저장 후 kakao api 로 위도경도 
못받아온 경우 인덱스 확인후 drop 
페이지를 바꾸려면 셀레늄 활용 하고 BS로 클롤링 후 지도에표시

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
driver = webdriver.Chrome()
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=35.1469568&myY=126.9235712&searchKeyword='
driver.get(url)

In [2]:
# 입력창에 서울입력 하고 검색 버튼을 누르는 동작
driver.find_element(By.ID, "searchKeyword").send_keys('서울')
driver.find_element(By.ID, "fsSearchBtn").click()

In [3]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [5]:
li = lis[0]
name = li.select_one('.title').get_text().strip()
addr = li.find('address').get_text().strip()
tel = li.select_one('.tel').get_text().strip()

In [6]:
name, addr, tel 

('동작소방서', '서울특별시 동작구 여의대방로16길 55(신대방동)', '02-847-1190')

In [7]:
lines = []
for li in lis:
    name = li.select_one('.title').get_text().strip()
    addr = li.find('address').get_text().strip()
    tel = li.select_one('.tel').get_text().strip()
    lines.append([name, addr, tel])

In [8]:
lines

[['동작소방서', '서울특별시 동작구 여의대방로16길 55(신대방동)', '02-847-1190'],
 ['서대문소방서', '서울특별시 서대문구 연희로 182(연희동)', '02-3144-1190'],
 ['광진소방서', '서울특별시 광진구 광나루로 480(구의동)', '02-457-0119'],
 ['송파소방서', '서울특별시 송파구 오금로51길 56(마천동)', '02-403-2119'],
 ['양천소방서', '서울특별시 양천구 목동서로 180(목동)', '02-2655-1119'],
 ['은평소방서', '서울특별시 은평구 통일로 962(진관동)', '02-355-0119'],
 ['노원소방서', '서울특별시 노원구 한글비석로 1길 8(하계동)', '02-977-0119'],
 ['종로소방서', '서울특별시 종로구 종로1길 28(수송동)', '02-735-6119'],
 ['도봉소방서', '서울특별시 도봉구 도봉로 666(방학동)', '02-3492-3437'],
 ['용산소방서', '서울특별시 용산구 한강대로 167(한강로2가)', '02-794-0119']]

# 두번째 페이지로 이동

In [13]:
# 1페이지에서 다음페이지를 누르면 2페이지 아님
# 3페이지가 나옴 그부분을 if 으로 처리행 함
driver.find_element(By.XPATH, 
'//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()

In [12]:
driver.find_element(By.CSS_SELECTOR, '.next_page').click()

In [15]:
# 검색결과 건수 획득
strong = soup.select('.stations-local-after > p > strong')[1].get_text()
strong

'100건'

In [16]:
count = int(strong[1].get_text().replace('건', '').strip())
count

100

# 전체 코드 정리

In [21]:
driver = webdriver.Chrome()
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=35.1469568&myY=126.9235712&searchKeyword='
driver.get(url)
time.sleep(2)
driver.find_element(By.ID, "searchKeyword").send_keys('서울')
driver.find_element(By.ID, "fsSearchBtn").click()
time.sleep(2)

In [22]:
import math
strong = soup.select('.stations-local-after > p > strong') 
count = int(strong[1].get_text().replace('건', '').strip())
count = math.ceil(count / 10)
count

10

In [23]:
# a = 101
# math.ceil(a / 10)

In [24]:
lines = [] 
for i in range(count):
    if i >= 1 and i % 2 == 0:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(4)
    if i >= 1 and i % 2 == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(4)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')   
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.find('address').get_text().strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append([name, addr, tel])

In [25]:
len(lines)

100

In [26]:
driver.close()

In [27]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '전화번호'])
df.tail()

,관서명,주소,전화번호
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119
99,송파소방서-마천-119 안전센터,서울특별시 송파구 오금로51길 56 (마천동),02-3401-2119


In [28]:
df.to_csv('서울소방서.csv', index=False)